In [130]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import yfinance as yf

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [131]:
tf.__version__

'2.13.0'

In [132]:
class AI_Trader():

    def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
                
        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque (maxlen=2000)
        self.inventory = []
        self.model_name = model_name

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995

        self.model = self.model_builder()

    def model_builder(self):
    
        model = tf.keras.models.Sequential()

        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))

        model.add(tf.keras.layers.Dense(units=64, activation='relu'))

        model.add(tf.keras.layers.Dense(units=128, activation='relu'))

        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))

        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))

        return model 
        
    def trade(self, state):

        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
    
        actions = self.model.predict(state)
        return np.argmax(actions[0])


    def batch_train(self, batch_size):
    
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])
      
        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
            target = self.model.predict(state)
            target[0][action] = reward
      
            self.model.fit(state, target, epochs=1, verbose=0)
      
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

In [133]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [134]:
def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [135]:
def dataset_loader(stock_name):
    stock_data = yf.download(stock_name)
    close = stock_data['Close']
    return close

# Call the dataset_loader function with the desired stock symbol
stock_symbol = "AAPL"
close_data = dataset_loader(stock_symbol)
print(close_data)

[*********************100%%**********************]  1 of 1 completed
Date
1980-12-12      0.128348
1980-12-15      0.121652
1980-12-16      0.112723
1980-12-17      0.115513
1980-12-18      0.118862
                 ...    
2023-08-18    174.490005
2023-08-21    175.839996
2023-08-22    177.229996
2023-08-23    181.119995
2023-08-24    176.380005
Name: Close, Length: 10765, dtype: float64


In [136]:
def state_creator(data, timestep, window_size):

    starting_id = timestep - window_size + 1

    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])

    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

    return np.array ([state])

In [137]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

[*********************100%%**********************]  1 of 1 completed


In [138]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data - 1)

In [139]:
trader = AI_Trader(window_size)

In [140]:
for episode in range (1, episodes + 1):

    print("Episode: {}/{}".format(episode, episodes))

    state = state_creator(data, 0, window_size + 1)

    total_profit = 0
    trader.inventory = []
    
    for t in tqdm(range(data_samples)):
        
        action = trader.trade(state)

        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0

        if action == 1: #Buying
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stocks_price_format (data[t]))

        elif action == 2 and len(trader.inventory) > 0: #Selling
            buy_price = trader.inventory.pop(0)

            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )

        if t == data_samples - 1:
            done = True
        else:
            done = False
    
        trader.memory.append((state, action, reward, next_state, done))

        state = next_state

        if done:
            print("###########################")
            print("TOTAL PROFIT: {}".format(total_profit))
            print("###########################")

        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)

    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))

Episode: 1/1000


  0%|          | 0/10765 [00:00<?, ?it/s]

AI Trader bought:  $ 0.128348
AI Trader sold:  $ 0.121652  Profit: - $ 0.006696
AI Trader bought:  $ 0.112723
AI Trader sold:  $ 0.115513  Profit: $ 0.002790
AI Trader bought:  $ 0.118862
AI Trader bought:  $ 0.126116
AI Trader bought:  $ 0.132254
AI Trader sold:  $ 0.137835  Profit: $ 0.018973
AI Trader sold:  $ 0.145089  Profit: $ 0.018973
AI Trader sold:  $ 0.158482  Profit: $ 0.026228
AI Trader bought:  $ 0.160714
AI Trader bought:  $ 0.156808
AI Trader sold:  $ 0.152344  Profit: - $ 0.008370
AI Trader sold:  $ 0.154018  Profit: - $ 0.002790
AI Trader bought:  $ 0.137835
AI Trader sold:  $ 0.135045  Profit: - $ 0.002790
AI Trader bought:  $ 0.136161
AI Trader bought:  $ 0.139509
AI Trader bought:  $ 0.138393
AI Trader sold:  $ 0.146763  Profit: $ 0.010602
AI Trader sold:  $ 0.142299  Profit: $ 0.002790
AI Trader sold:  $ 0.146763  Profit: $ 0.008370
AI Trader bought:  $ 0.146205
AI Trader bought:  $ 0.138393
AI Trader sold:  $ 0.133371  Profit: - $ 0.012834
1/1 [===================

  0%|          | 33/10765 [00:08<47:18,  3.78it/s]

AI Trader sold:  $ 0.126116  Profit: - $ 0.012277
1/1 [==============================] - 0s 35ms/step


...

1/1 [==============================] - 0s 43ms/step
AI Trader sold:  $ 0.126674  Profit: $ 0.003906
1/1 [==============================] - 0s 21ms/step


  1%|          | 96/10765 [07:27<32:19:24, 10.91s/it]

1/1 [==============================] - 0s 118ms/step


  1%|          | 97/10765 [07:36<30:51:18, 10.41s/it]

1/1 [==============================] - 0s 50ms/step
AI Trader sold:  $ 0.126116  Profit: $ 0.001674
1/1 [==============================] - 0s 258ms/step
